In [1]:
import numpy as np
import pandas as pd
from methods import list_files_in_directory, init_model, load_model
import torch
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
DEVICE = 'mps'

In [3]:
model_name = "01"
path_to_weights = (
    "/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-"
    + model_name
    + "/pattern/dc_weights.csv"
)


path_to_in_data_1 = (
    "/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-"
    + model_name
    + "/pattern/in_data_d1.csv"
)
path_to_in_data_2 = (
    "/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-"
    + model_name
    + "/pattern/in_data_d2.csv"
)
path_to_in_data_3 = (
    "/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-"
    + model_name
    + "/pattern/in_data_d3.csv"
)

In [4]:
weights = torch.from_numpy(pd.read_csv(path_to_weights, header=None).values.T).to(
    DEVICE, dtype=torch.float32
)
state_size = len(weights)

In [5]:
data_in_all_list = [pd.read_csv(path_to_in_data, header=None).values for path_to_in_data in [path_to_in_data_1, path_to_in_data_2, path_to_in_data_3]]
data_in_all_array = np.vstack(data_in_all_list)

In [6]:
BATCH_SIZE = len(data_in_all_array)
N_STEPS = 40
FC2_LENGTH = 128

In [7]:
all_models_path = '/Users/hazimiasad/Documents/Work/megan/code/playground/RL-Diffusion/results/models/sub-'+model_name

In [8]:
all_models = list_files_in_directory(all_models_path)

In [9]:
model = init_model(DEVICE, state_size, state_size)
model, model_name = load_model(model, all_models[0])
model = model.to(DEVICE)

In [10]:
policy_network = model.policy

In [11]:
activations = np.zeros((BATCH_SIZE, N_STEPS, FC2_LENGTH))

def get_activation(data_idx, step_idx):
    def hook(model, input, output):
        activations[data_idx, step_idx] = output.cpu().detach()  # Store activation without gradient computation
    return hook

In [ ]:
for x_idx, x in tqdm(enumerate(data_in_all_array), total=len(data_in_all_array)):
    x = torch.from_numpy(x.reshape(1, state_size)).float().to(DEVICE)
    for t in range(40, 0, -1):
        policy_network.fc2.register_forward_hook(get_activation(x_idx, t - 1))
        with torch.no_grad():
            mean, std = policy_network(x, t)
        dist = torch.distributions.Normal(mean, std)
        action = dist.sample().clamp(-1.0, 1.0)
        x = x + action

 24%|██▍       | 111/465 [31:32<3:12:38, 32.65s/it]

In [ ]:
data_in_all_array[0, :].shape

In [ ]:
activations['fc2'].shape

In [ ]:
plt.imshow(activations['fc2'].cpu())
plt.colorbar()
plt.show()